In [1]:
!pip install openai python-dotenv faiss-cpu tiktoken pdfplumber python-docx PyPDF2


In [2]:
import os
import numpy as np
import faiss
import json
from openai import AzureOpenAI

In [3]:
# Azure OpenAI endpoint and key
azure_openai_endpoint_model = "https://assessment2025-resource.cognitiveservices.azure.com/"
azure_openai_api_key_model = "EeK1erGEZA2b3ec4sDGzx3JvwUUvAv4YU6LbQ6gWJqKcJya33LK1JQQJ99BKACHYHv6XJ3w3AAAAACOGUL8L"
azure_endpoint_embedding = "https://assessment2025-resource.cognitiveservices.azure.com/"
azure_api_key_embedding = "EeK1erGEZA2b3ec4sDGzx3JvwUUvAv4YU6LbQ6gWJqKcJya33LK1JQQJ99BKACHYHv6XJ3w3AAAAACOGUL8L"


In [4]:




# Set the environment variables (alternatively, you can pass them directly to the client constructor)
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_openai_endpoint_model
os.environ["AZURE_OPENAI_KEY"] = azure_openai_api_key_model

os.environ["AZURE_OPENAI_MODEL"] = "gpt-4.1-mini"
os.environ["AZURE_OPENAI_EMBED"] = "text-embedding-3-small"
os.environ["AZURE_OPENAI_VERSION"] = "2024-02-01"


## Embedding


In [5]:
api_version = "2023-05-15"
deployment_name = "text-embedding-ada-002"

client1 = AzureOpenAI(
    api_key=azure_api_key_embedding,
    api_version=api_version,
    azure_endpoint=azure_endpoint_embedding
)


In [6]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_VERSION")
)

MODEL = os.getenv("AZURE_OPENAI_MODEL")
EMBED = os.getenv("AZURE_OPENAI_EMBED")


### Extract Text from PDF

In [7]:
import PyPDF2
def load_document(pdf_path):
    """
    Extract text from a PDF file.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        str: Extracted text from all pages
    """
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

### Embedding api used

In [10]:
def embed_text(text_list):
    response = client1.embeddings.create(
        model=deployment_name,
        input=text_list
    )
    vectors = [item.embedding for item in response.data]
    return np.array(vectors)


### Chunking text

In [11]:
def chunk_text(text, chunk_size=300):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks



### Create indexing using Faiss

In [13]:
import faiss

def create_index(document_text):

    chunks = chunk_text(document_text)
    vectors = embed_text(chunks)

    dim = vectors.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vectors)

    print(f"FAISS index contains {index.ntotal} vectors.")

    return index, chunks




8. Retrieval Function

In [15]:
def retrieve(query, k=3):
    q_emb = embed_text([query])
    distances, indices = index.search(q_emb, k)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    return retrieved_chunks


9. Agent Functions (Tool Caller + Critic + Final Answer)

9.1 Tool Calling (decide to use retrieval)

In [16]:
def decide_tool(query):
    msg = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "You decide if the retriever tool is needed. Answer YES or NO only."
            },
            {"role": "user", "content": query}
        ]
    )
    return msg.choices[0].message.content.strip()

9.2 Critic Model

In [17]:
def critic_analysis(query, retrieved):
    msg = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "Critic model. Check if retrieved evidence supports the answer."
            },
            {
                "role": "user",
                "content": f"User query: {query}\nRetrieved:\n{retrieved}"
            }
        ]
    )
    return msg.choices[0].message.content

9.3 Final Answer Generator

In [18]:
def final_answer(query, retrieved, critic_notes):
    msg = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "You are a RAG assistant. Use only retrieved chunks. Avoid hallucination."
            },
            {
                "role": "user",
                "content": f"Query: {query}\nRetrieved: {retrieved}\nCritic: {critic_notes}"
            }
        ]
    )
    return msg.choices[0].message.content

10. Main Agent Function

In [19]:

def agent(query):
    decision = decide_tool(query)

    if "YES" in decision:
        retrieved = retrieve(query)
    else:
        retrieved = []

    critic_notes = critic_analysis(query, retrieved)
    answer = final_answer(query, retrieved, critic_notes)

    return answer


11. Chat Loop Cell (command-line inside notebook)

In [ ]:
# user insert document file path:
print("Please insert  the  document  file  path:")
path = input()
document_text = load_document(path)
index, chunks = create_index(document_text)
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break

    print("\nAnswering...\n")
    response = agent(query)
    print("Assistant:", response, "\n")


Please insert  the  document  file  path:
/content/attention.pdf
FAISS index contains 20 vectors.
You: What is attention mechanism?

Answering...

Assistant: The attention mechanism is a concept used in machine learning and neural networks, especially in natural language processing. It allows a model to focus on different parts of the input sequence when generating each part of the output sequence. This dynamic weighting of input elements helps the model capture relevant context and relationships within the data more effectively, improving performance on tasks such as translation, summarization, and image captioning. 

You: What is attention mechanism in transformer.

Answering...

Assistant: The retrieved information does not contain details about the attention mechanism in transformers. Based on general knowledge, the attention mechanism in transformers is a process that allows the model to weigh the importance of different words in a sequence when making predictions, enabling it to 

What is attention mechanism in transformer.


In [ ]:
What is the model architecture of Transformer?
What is attention mechanism in transformer.
What is attention mechanism?
What is the Transformer architecture?